In [123]:
from datetime import datetime, time, timedelta
from time import sleep, gmtime, strftime
from dateutil import parser
import holidays
import pytz

In [124]:
def sleep_till_open():

    """
    Checks for the next day the U.S. Stock Market is open based on Eastern Standard Time (EST). 
    The function then sleeps the program from the current datetime 
    until the market is open at 9:30am.
    """

    # Insert your timezone into the timezone function
    eastern_tz = pytz.timezone('US/Eastern')
    current_time_eastern = datetime.now(eastern_tz).isoformat()
    nyse = holidays.NYSE()
    next_open = nyse.get_nth_working_day(current_time_eastern, 1)
    next_trading_day = datetime.combine(next_open, time(9, 30)).isoformat()
    
    # New additon to print the datetime of the next open
    date_diff = eastern_tz.localize(datetime.combine(next_open, time(9, 30))) - datetime.now(eastern_tz)

    # Format the date_diff into a readable string with days, months, and time
    days_remaining = date_diff.days
    hours, remainder = divmod(date_diff.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Construct the output string
    formatted_diff = f"Market will open in {days_remaining} days, {hours} hours, {minutes} minutes, and {seconds} seconds"
    print(formatted_diff)
    while datetime.now() < parser.parse(next_trading_day):
        sleep(1)

In [125]:
def market_is_open():
    
    """
    Checks whether the U.S. stock market (NYSE) is currently open based on Eastern Standard Time (EST).
    This function considers both the current date and time, taking into account:
    - U.S. holidays observed by NYSE.
    - Weekends (Saturday and Sunday) when the market is closed.
    - Market open hours (9:00 AM to 4:00 PM EST) on weekdays when the market is open.
    """

    try:
        # Set the timezone to your timezone. My timezone is Eastern Standard Time (EST)
        eastern_tz = pytz.timezone('US/Eastern')
        current_time_eastern = datetime.now(eastern_tz)

        # Check if the current day a recognized holiday
        nyse_holidays = holidays.NYSE()
        today_date_str = current_time_eastern.strftime('%Y-%m-%d')
        is_holiday = nyse_holidays.get(today_date_str)
        if is_holiday:
            sleep_till_open()
            market_is_open()

        # Check if the current day (EST) is a weekday
        day_of_week = current_time_eastern.weekday()
        if day_of_week == 5 or day_of_week == 6:
            sleep_till_open()
            market_is_open()
            

        # Define the market open and close times in EST
        market_open_time = time(9, 30)  # Market opens at 9:00 AM EST for premarket orders and 9:30 AM EST for all other orders
        market_close_time = time(16, 0)  # Market closes at 4:00 PM EST

        if market_open_time <= current_time_eastern.time() <= market_close_time:
            return True
        else:
            sleep_till_open()
            market_is_open()

    except Exception as err:
        print(err)

    return True

In [126]:
market_is_open()

Market will open in 0 days, 17 hours, 2 minutes, and 39 seconds


KeyboardInterrupt: 